In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import re

sys.path.append("../")
warnings.filterwarnings("ignore")

In [ ]:
import os


root_dir = os.path.abspath('../bailian_nlp/')

data_path = os.path.join(root_dir, 'datadrive/bailian/pos')
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

train_path = os.path.join(data_path, 'train.csv')
valid_path = os.path.join(data_path, 'valid.csv')

model_dir = os.path.join(root_dir, 'datadrive/models/chinese_L-12_H-768_A-12/')
init_checkpoint_pt = os.path.join(model_dir, 'bert_model.bin')
bert_config_file = os.path.join(model_dir, 'bert_config.json')
vocab_file = os.path.join(model_dir, 'vocab.txt')
model_pt = os.path.join(model_dir, 'pos.bin')
config_file = os.path.join(model_dir, 'pos.json')



In [3]:
# 初始化构造数据
from bailian_nlp.preprocessing.build_data import build_pos_fake_data, build_pos_train_and_valid_data
# build_pos_fake_data()
build_pos_train_and_valid_data()



530 165566 0.003201140330744235
531 165711 0.0032043738798269274
532 165712 0.003210389108815294
533 165713 0.003216404265205506
534 166055 0.003215801993315468
535 166060 0.0032217270865952065
536 166749 0.0032144120804322666
537 167083 0.0032139714991950108
538 167553 0.0032109243045484116
539 167569 0.003216585406608621
540 170842 0.0031608152562016367
541 171156 0.0031608590993012224
542 171191 0.0031660542902372203
543 171338 0.0031691743804643452
544 171339 0.0031749922667927326
545 171340 0.0031808100852106923
546 171375 0.003185995623632385
547 171376 0.0031918121557277567
548 171377 0.0031976286199431663
549 171400 0.0032030338389731623
550 172290 0.003192292065703175
551 172772 0.003189174171740791
552 172791 0.0031946108304251957
553 174101 0.003176317195191297
554 174106 0.003181969604723559
555 174468 0.0031810991127312746
556 174485 0.0031865203312605667
557 174486 0.003192233187762915
558 174487 0.0031979459787835197
559 174491 0.0032036036242556925
560 174847 0.00320280

UnsupportedOperation: not readable

In [ ]:
# 正常训练

from bailian_nlp.modules import BertData

data = BertData.create(
    train_path,
    valid_path, 
    vocab_file,
    data_type="bert_uncased",
    max_seq_len=128,
    batch_size=128
    
)

import torch
import torch.nn as nn
from importlib import reload
from bailian_nlp.modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    enc_hidden_dim=256
)


from bailian_nlp.modules.train import train
reload(train)
num_epochs = 10
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')


In [ ]:
import torch
import torch.nn as nn
from importlib import reload
from bailian_nlp.modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    enc_hidden_dim=256
)
model.get_n_trainable_params()


from bailian_nlp.modules.train import train
reload(train)
num_epochs = 2
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')

In [ ]:
# 恢复训练

from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()
tagger.init_env(for_train=True)

data = tagger.learner.data
learner = tagger.learner
num_epochs = 1
learner.load_model()
learner.t_total = num_epochs * len(data.train_dl)
learner.sup_labels = list(set(data.id2label[1:]) | set(learner.sup_labels))
learner.fit(num_epochs, target_metric='f1')


In [ ]:
learner.fit(num_epochs, target_metric='f1')

In [ ]:
from modules.data import bert_data
reload(bert_data)
dl = bert_data.get_bert_data_loader_for_predict(valid_path, learner)

In [ ]:
# learner.load_model()
preds = learner.predict(dl)

In [ ]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds)
print(clf_report)

In [ ]:
from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()


In [ ]:
import time

st = time.time()
text = '近日，编程猫（深圳点猫科技有限公司）正式对外宣布完成B轮1.2亿元融资。本轮融资由高瓴资本领投，清流资本、清晗基金跟投，天使轮投资者猎豹移动继续跟投。'
# text = '未来编程教育产业将蓬勃发展，编程猫作为提供工具与内容的企业，有望长期处于行业领跑者地位。'
# text = '美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司始创于2004年,是中国健康体检和医疗服务集团,总部位于上海,深耕布局北京、深圳、沈阳、广州、成都、武汉、...'
# text = '百炼智能百炼智能'
# text = '高越君冯是聪'
text = '周光明确否认CEO佟显侨和衡量推动发出公司公告'
text = '周光明确否认CEO佟显侨和CTO衡量说的罪名'
text = '董事'
# text = '一言九鼎'
# text = '客户包括雀巢、洲际酒店、瑞士航空、德意志银行、红牛、瑞士联合银行等世界知名公司。'
# text = '药方越是多的，越表明病是难的于治疗'
text = '2012-11-8 的客户包括：宝洁 、杜邦、强生、百事可 乐、3M 、三菱、吉列、通用医疗系统、美国 运通、雅芳、纽约银 行、百时美—施贵 宝、礼来、迪斯尼、纳贝斯克、纽约'
res = tagger.cut(text)
ed = time.time()
print(ed - st)
res